In [173]:
import pandas as pd
import plotly.express as px
from pandas.api.types import CategoricalDtype

#import the csvs
medicalDoctors = pd.read_csv('medicalDoctors.csv')
nursingAndMidwife = pd.read_csv('nursingAndMidwife.csv')
Dentists = pd.read_csv('dentists.csv')
Pharmacists = pd.read_csv('pharmacists.csv')
basicHandWashing = pd.read_csv('basicHandWashing.csv')
atLeastBasicSanitizationServices = pd.read_csv('atLeastBasicSanitizationServices.csv')
basicDrinkingWaterServices = pd.read_csv('basicDrinkingWaterServices.csv')
lifeExpectancyAtBirth = pd.read_csv('lifeExpectancyAtBirth.csv')
under5MortalityRate = pd.read_csv('under5MortalityRate.csv')



In [174]:
#Takes all the data frames and renames the columns so they don't all have the same name
def column_renamer(df_list):

    #Iterate through the list
    for i in range(len(df_list)):
        df = df_list[i]

        #Check that the 'Indicator' column isn't actually useful
        if len(df['Indicator'].drop_duplicates()) == 1:
            column_name = df['Indicator'].iloc[0]
            df = df.rename(columns={"First Tooltip": column_name})
            df.drop('Indicator', axis=1, inplace=True)
            df_list[i] = df  # Save the updated data frame back into the list
    return df_list


#run data frames through method
df_list = [medicalDoctors, nursingAndMidwife, Dentists, Pharmacists, basicHandWashing, atLeastBasicSanitizationServices, basicDrinkingWaterServices,
        lifeExpectancyAtBirth, under5MortalityRate]
df_list = column_renamer(df_list)

(medicalDoctors, nursingAndMidwife, Dentists, Pharmacists, basicHandWashing, atLeastBasicSanitizationServices, basicDrinkingWaterServices,
  lifeExpectancyAtBirth, under5MortalityRate) = df_list #Updates data frames

In [175]:
#Look for a pattern among the distribution of doctors and life expectancy
doctors = medicalDoctors.merge(nursingAndMidwife, on = ['Location', 'Period'], how = 'outer')
doctors = doctors.merge(Dentists, on = ['Location', 'Period'], how = 'outer')
doctors = doctors.merge(Pharmacists, on = ['Location', 'Period'], how = 'outer')


#Group doctors and pharmacists into categorical variables
doctors['RateDoctors'] = doctors['Medical doctors (per 10,000)'].apply( lambda rate: 'low' if rate < 20 else ('high' if rate > 30 else 'medium'))
doctors['RatePharmacists'] = doctors['Pharmacists  (per 10,000)'].apply( lambda rate: 'low' if rate < 2 else ('high' if rate >= 6 else 'medium'))

#Plot effect of doctors on life expectancy
lifeExpectancyAtBirth = lifeExpectancyAtBirth.loc[lifeExpectancyAtBirth['Dim1'] == 'Both sexes'] #Remove gendered classifiers
doctors_life = doctors.merge(lifeExpectancyAtBirth, on = ['Location', 'Period'], how = 'inner')

# Group by labels
doctors_life_mean = doctors_life.groupby(['RateDoctors', 'RatePharmacists'])['Life expectancy at birth (years)'].mean().reset_index()
doctors_life_mean['Group'] = (doctors_life_mean['RatePharmacists'] + ' pharmacists ' + doctors_life_mean['RateDoctors'] + ' doctors')

#Change colors so plot is easier to read
doctors_life_mean['Color Group'] = doctors_life_mean['Life expectancy at birth (years)'].apply(lambda x: 'Life Expectancy Under 70' if x < 70 
                                                                                               else 'Life Expectancy Above 70')
custom_colors = {'Life Expectancy Under 70': 'purple', 'Life Expectancy Above 70': 'gray'}

# Plot
fig = px.bar(doctors_life_mean, x='Group', y='Life expectancy at birth (years)', title=f"Doctors' Effect on Life Expectancy", category_orders={'Group': [
        'high pharmacists high doctors',
        'high pharmacists medium doctors',
        'high pharmacists low doctors',
        'medium pharmacists high doctors',
        'medium pharmacists medium doctors',
        'medium pharmacists low doctors',
        'low pharmacists high doctors',
        'low pharmacists medium doctors',
        'low pharmacists low doctors'
    ]}, color = 'Color Group', color_discrete_map=custom_colors, labels={'Life expectancy at birth (years)'}, text_auto=True)
fig.update_traces(textfont_color='white')
fig.show()

Question first plot answers: How important is doctor and drug avaliability to overall life expectancy? 10 year life expectancy difference in countries without access to doctors, showing the importance of charities like Doctors Without Borders and Samaritan's Purse. Drug avaliability is also incredibly important: despite having a high amount of doctors per population, average life expectancy is hovering just over 70 in countries with few pharmacists. In countries that have few doctors but high amounts of pharmacists, life expectancy on average is 77 years.

Statistics: Find if the number of doctors makes a significant difference vs sanitation if life expectancy

In [ ]:
#Find prevelance of basic sanitation services
total_basicHandWashing = basicHandWashing.loc[basicHandWashing['Dim1'] == 'Total'].copy() #Remove 'Urban' and 'Rural' classifiers
total_basicHandWashing.drop('Dim1', axis=1, inplace=True)
cleanliness = basicDrinkingWaterServices.merge(total_basicHandWashing, on = ['Location', 'Period'], how = 'outer')

#Clean child mortality data
under5MortalityRate = under5MortalityRate.loc[under5MortalityRate['Dim1'] == 'Both sexes'].copy()
under5MortalityRate.drop('Dim1', axis=1, inplace=True)
under5MortalityRate['Children Dying by Age 5 per 1,000 Live Births'] = under5MortalityRate['Under-five mortality rate (probability of dying by age 5 per 1000 live births)'].apply(lambda x: x.partition("[")[0]).copy()
under5MortalityRate['Children Dying by Age 5 per 1,000 Live Births'] = pd.to_numeric(under5MortalityRate['Children Dying by Age 5 per 1,000 Live Births'])
under5MortalityRate.drop('Under-five mortality rate (probability of dying by age 5 per 1000 live births)', axis=1, inplace=True)


#Effects of handwashing and sanitary services on child mortality
cleanliness_life = cleanliness.merge(under5MortalityRate, on = ['Location', 'Period'], how = 'inner')
cleanliness_life['HandWashing'] = cleanliness_life['Population with basic handwashing facilities at home (%)'].apply( lambda rate: 'low' if rate < 50 else 'high')

#Plot
px.scatter(cleanliness_life, x = 'Population using at least basic drinking-water services (%)', y = 'Children Dying by Age 5 per 1,000 Live Births', color = 'HandWashing', opacity = 0.5).show()



#1995, WHO starts tackling child illnesses. Timeline of WHO's actions will hopefully correspond to what we see on the visualization: 
# https://www.who.int/campaigns/75-years-of-improving-public-health/milestones#year-1998

         Location  Period  \
0     Afghanistan    2000   
1     Afghanistan    2001   
2     Afghanistan    2002   
3     Afghanistan    2003   
4     Afghanistan    2004   
...           ...     ...   
3055  Timor-Leste    2013   
3056  Timor-Leste    2014   
3057  Timor-Leste    2015   
3058  Timor-Leste    2016   
3059  Timor-Leste    2017   

      Population using at least basic drinking-water services (%)  \
0                                                 21.62             
1                                                 21.62             
2                                                 23.60             
3                                                 25.58             
4                                                 27.56             
...                                                 ...             
3055                                              61.07             
3056                                              63.18             
3057                          

Lack of clean water sources has the biggest impact on children. According to WHO (https://www.who.int/publications/i/item/9789240103412) 
pneumonia and diarrhoea accounted for 23% of deaths among children under 5. Pneumonia can be reduced with access and to handawashing services, and diarrhoea is mainly caused by uncleaning drinking water. Compassion International provides septic systems and sanitation equipment to prevent the spread of infectious diseases and World Vision digs wells.